# Выбор локации для скважины


компании «ГлавРосГосНефть», нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Нжно построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. 

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
state = np.random.RandomState(12345)
from scipy import stats as st

Импортируем таблицы для работы в соответствующие переменные

In [2]:
data0 = pd.read_csv('/datasets/geo_data_0.csv')
data1 = pd.read_csv('/datasets/geo_data_1.csv')
data2 = pd.read_csv('/datasets/geo_data_2.csv')

И просмотрим их

In [3]:
data0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
data1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
data2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Посмотрю на уникальные id, в идеале там не должно быть повторов

In [6]:
len(data0['id'].unique())

99990

меньше, чем общее кол-во строк, тогда смотрю дубликаты

In [7]:
data0[data0['id'].duplicated()]

,id,f0,f1,f2,product
7530,HZww2,1.061194,-0.373969,10.430210,158.828695
41724,bxg6G,-0.823752,0.546319,3.630479,93.007798
51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205
63593,QcMuo,0.635635,-0.473422,0.862670,64.578675
66136,74z30,1.084962,-0.312358,6.990771,127.643327
69163,AGS9W,-0.933795,0.116194,-3.655896,19.230453
75715,Tdehs,0.112079,0.430296,3.218993,60.964018
90815,fiKDv,0.049883,0.841313,6.394613,137.346586
92341,TtcGQ,0.110711,1.022689,0.911381,101.318008
97785,bsk9y,0.378429,0.005837,0.160827,160.637302


In [8]:
data0[data1['id'].duplicated()]

,id,f0,f1,f2,product
41906,hvrg6,1.356490,-0.418635,1.296605,88.635132
82178,1mHHg,0.120088,-0.236548,-0.160710,86.774162
82873,RFXea,-0.892442,0.367704,3.921657,75.949034
84461,ubDZy,0.688830,0.800470,-0.547005,57.522227


In [9]:
data0[data2['id'].duplicated()]

,id,f0,f1,f2,product
43233,mPBNw,0.146706,1.164548,2.147357,12.219334
49564,tmiKi,1.469579,-0.367773,0.551081,63.688098
55967,U9TtX,1.767004,-0.101328,1.521255,149.114609
95090,15KPx,0.208177,0.934896,-2.127103,24.814889


Избавлюсь от них

In [10]:
data0 = data0.drop_duplicates(subset='id')
data1 = data1.drop_duplicates(subset='id')
data2 = data2.drop_duplicates(subset='id')

In [11]:
data0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99990 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       99990 non-null  object 
 1   f0       99990 non-null  float64
 2   f1       99990 non-null  float64
 3   f2       99990 non-null  float64
 4   product  99990 non-null  float64
dtypes: float64(4), object(1)
memory usage: 4.6+ MB


In [12]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99996 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       99996 non-null  object 
 1   f0       99996 non-null  float64
 2   f1       99996 non-null  float64
 3   f2       99996 non-null  float64
 4   product  99996 non-null  float64
dtypes: float64(4), object(1)
memory usage: 4.6+ MB


In [13]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99996 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       99996 non-null  object 
 1   f0       99996 non-null  float64
 2   f1       99996 non-null  float64
 3   f2       99996 non-null  float64
 4   product  99996 non-null  float64
dtypes: float64(4), object(1)
memory usage: 4.6+ MB


In [14]:
data0.duplicated().sum()

0

In [15]:
data1.duplicated().sum()

0

In [16]:
data2.duplicated().sum()

0

Еще взгляну на распределение признаков

In [17]:
data0.describe()

,f0,f1,f2,product
count,99990.000000,99990.000000,99990.000000,99990.000000
mean,0.500454,0.250141,2.502629,92.499684
std,0.871844,0.504430,3.248149,44.288304
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072572,-0.200877,0.287784,56.497069
50%,0.502405,0.250252,2.515969,91.847928
75%,1.073626,0.700640,4.715035,128.563699
max,2.362331,1.343769,16.003790,185.364347


Дубликатов, как и пропусков(вывод с info) нет

In [18]:
data1.describe()

,f0,f1,f2,product
count,99996.000000,99996.000000,99996.000000,99996.000000
mean,1.141209,-4.796608,2.494501,68.823916
std,8.965815,5.119906,1.703579,45.944663
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011475,57.085625
75%,8.620964,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [19]:
data2.describe()

,f0,f1,f2,product
count,99996.000000,99996.000000,99996.000000,99996.000000
mean,0.002002,-0.002159,2.495084,94.998342
std,1.732052,1.730397,3.473482,44.749573
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162328,-1.174841,0.130269,59.450028
50%,0.009424,-0.009661,2.484236,94.925026
75%,1.158477,1.163523,4.858720,130.586815
max,7.238262,7.844801,16.739402,190.029838


В описании данных нет пояснения к  f, да и понять по этим данным сложно, но тк В ТЗ сказано, они важны - не трогаю. Причем эти признаки распределны разным способом, только среднее значение в f2 практический совподает, product имеет больше схожестей

Если product - целевой признак, то меня интересует корреляция признаков с ним

In [20]:
data0.corr()

,f0,f1,f2,product
f0,1.000000,-0.440717,-0.003211,0.143504
f1,-0.440717,1.000000,0.001764,-0.192351
f2,-0.003211,0.001764,1.000000,0.483631
product,0.143504,-0.192351,0.483631,1.000000


Сильной зависимости нет

In [21]:
data1.corr()

,f0,f1,f2,product
f0,1.000000,0.182279,-0.001810,-0.030523
f1,0.182279,1.000000,-0.002594,-0.010154
f2,-0.001810,-0.002594,1.000000,0.999397
product,-0.030523,-0.010154,0.999397,1.000000


тут есть очень сильная зависимость от f2

In [22]:
data2.corr()

,f0,f1,f2,product
f0,1.000000,0.000506,-0.000452,-0.001978
f1,0.000506,1.000000,0.000753,-0.001055
f2,-0.000452,0.000753,1.000000,0.445867
product,-0.001978,-0.001055,0.445867,1.000000


Как и в 1 случае, тут без сильной зависимости

**Итог предобработки**

- Удалил совподающие id
- Других проблем(дубикатов, пропусков, типы данных и тд) не обнаружены
- Рассмотрел корреляцию, в одном случае обнаружил очень сильную зависимость, в других случаях ничего интресного
- Распределения признаков для меня остается загадкой

## Обучение и проверка модели

В ТЗ сказано:

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).

Поэтому занимаюсь ей

Тк нас интересует дискретная непрерывая переменная, задача является регрессионной

Прежде, чем обучать модель нужно удостовериться в правильности признаков, например масштабировать и привести все к численным значениям(в данном случае не требуется)

In [23]:
scaler = StandardScaler()

Разделять, масштабировать и обучать буду на разных моделях для каждого датафрейма тк данные за разные регионы и то что модель может счеть нормой для первого региона может не подойти для второго региона

In [24]:
target0 = data0['product']
features0 = data0.drop('product', axis=1)
features_train0, features_valid0, target_train0, target_valid0 = train_test_split(
    features0, target0, test_size=0.25, random_state=12345)

In [25]:
numeric = ['f0', 'f1', 'f2']
scaler.fit(features_train0[numeric])

pd.options.mode.chained_assignment = None 
features_train0.loc[:,numeric] = scaler.transform(features_train0.loc[:,numeric])
features_valid0.loc[:,numeric] = scaler.transform(features_valid0.loc[:,numeric])
print(features_train0.shape)
features_valid0.shape

(74992, 4)


(24998, 4)

In [26]:
target1 = data1['product']
features1 = data1.drop('product', axis=1)
features_train1, features_valid1, target_train1, target_valid1 = train_test_split(
    features1, target1, test_size=0.25, random_state=12345)

In [27]:
scaler.fit(features_train1[numeric])
 
features_train1.loc[:,numeric] = scaler.transform(features_train1.loc[:,numeric])
features_valid1.loc[:,numeric] = scaler.transform(features_valid1.loc[:,numeric])
print(features_train1.shape)
features_valid1.shape

(74997, 4)


(24999, 4)

In [28]:
target2 = data2['product']
features2 = data2.drop('product', axis=1)
features_train2, features_valid2, target_train2, target_valid2 = train_test_split(
    features2, target2, test_size=0.25, random_state=12345)

In [29]:
scaler.fit(features_train2[numeric])
 
features_train2.loc[:,numeric] = scaler.transform(features_train2.loc[:,numeric])
features_valid2.loc[:,numeric] = scaler.transform(features_valid2.loc[:,numeric])
print(features_train2.shape)
features_valid2.shape

(74997, 4)


(24999, 4)

Можно присутпать к обучению

In [30]:
model0 = LinearRegression()
model1 = LinearRegression()
model2 = LinearRegression()

Откинем груз

In [31]:
features_train0 = features_train0.drop('id', axis=1)
features_train1 = features_train1.drop('id', axis=1)
features_train2 = features_train2.drop('id', axis=1)

features_valid0 = features_valid0.drop('id', axis=1)
features_valid1 = features_valid1.drop('id', axis=1)
features_valid2 = features_valid2.drop('id', axis=1)

In [32]:
model0.fit(features_train0, target_train0)
model1.fit(features_train1, target_train1)
model2.fit(features_train2, target_train2)

LinearRegression()

Предсказываем

In [33]:
predict0 = model0.predict(features_valid0)
predict1 = model1.predict(features_valid1)
predict2 = model2.predict(features_valid2)

In [34]:
RMSE0 =  mean_squared_error(target_valid0, predict0) ** 0.5
print(RMSE0)
RMSE1 =  mean_squared_error(target_valid1, predict1) ** 0.5
print(RMSE1)
RMSE2=  mean_squared_error(target_valid2, predict2) ** 0.5
print(RMSE2)

37.853527328872964
0.8920592647717033
40.07585073246016


In [35]:
print(data0['product'].mean())
print(data1['product'].mean())
data2['product'].mean()

92.49968421774354
68.82391591804064


94.99834211933378

RMSE у второго региона чуть ли не идеальный, скорее всего это вызвано высоким уровнем кореляции одного из признаков с целевым признаком

**Итог**

- У нас явная задача регресси
- Для обучения была выбрана линейная регрессия
- Я обучил 3 разныз модели на каждый датафрейм для обеспечения болле точных результатов
- У первого и третьего региона близкие значения RMSE
- У втрого региона 2 особенности, один из признаков имеет выскокий уровень корреляции, а также обладает близким к 0 RMSE, что показывает повышенную точность модели для этого региона

## Подготовка к расчёту прибыли

Константы для расчетов из условия

In [36]:
RESEARCH_POINTS = 500  # количество исследуемых точек при разведке

TOP_RESEARCH_POINTS = 200  # количество лучших исследуемых точек 

BUDGET = 10 * 1e9  # бюджет на разработку скважин в регионе

PRODUCT_PRICE = 450_000 # цена за 1000 баррелей сырья

MAX_LOSS = 0.025 # максимальная вероятность убытков

BOOTSTRAP_SAMPLES = 1000  # количество выборок для bootstrap 

CONF_INTERVAL = 0.95 # доверительный интервал 

Сначала расчитаю budget_per_point - цена за одну точку, ведь по условию нужно найти топ 200 выгодных точек

А потом вычисчяю нужное минимальное кол-во для прибыли

In [37]:
budget_per_point = BUDGET / TOP_RESEARCH_POINTS
min_product = budget_per_point / PRODUCT_PRICE
print(min_product)

111.11111111111111


Минимальный объем требуемый для прибыли больше, чем средние значения во всех регионах, значит существует много точек где не выйдет добиться окупаемости

## Расчёт прибыли и рисков 

Напишу метод расчета прибыли для каждого региона, при условии, что для расчета будут взять первые 200 скважин имеющих больший запас при предсказаний, но значения реальные, то есть по индексам predict возьму target

In [38]:
def profit (predict, target):
    predict_S = pd.Series(predict).sort_values(ascending=False)
    target_S = (target.reset_index(drop = True)[predict_S.index])
    sum_true = target_S.head(200).sum() 
    return sum_true * PRODUCT_PRICE - BUDGET

In [39]:
profit(predict0,target_valid0)

3365187237.700287

In [40]:
profit(predict1,target_valid1)

2415086696.681511

In [41]:
profit(predict2,target_valid2)

2501283853.2820625

Выделив 200 самых прибыльных точек, можно добиться окупаемости в каждом из регионов, но больше всего в 1. Нужно проверить еще насколько случайны эти данные

In [42]:

def calc_price(predict,target_valid):
    values = []
    for i in range(BOOTSTRAP_SAMPLES):
        Predict0 = pd.Series(predict)
   
        subsample = Predict0.sample(500, replace=True, random_state=state)
        values.append(profit(subsample, target_valid))
    values = pd.Series(values)
    lower  = values.quantile(0.025)
    upper = values.quantile(0.975)
    risk = sum(values < 0) / len(values)
    print('Доверительный интервал:', lower, '-', upper)
    print('Средняя прибыль', values.mean())
    print('Риск убытка', risk)
calc_price(predict0,target_valid0)

Доверительный интервал: -142942739.74041605 - 890976833.8485551
Средняя прибыль 380613470.0175977
Риск убытка 0.077


In [43]:
calc_price(predict1,target_valid1)

Доверительный интервал: 73849212.0224793 - 917953817.7221825
Средняя прибыль 479159869.981588
Риск убытка 0.006


In [44]:
calc_price(predict2,target_valid2)

Доверительный интервал: -216265050.0341763 - 811060708.2365634
Средняя прибыль 315441916.0234899
Риск убытка 0.124


**Итог**

Самый прибыльный варинт - регион номер 2, в нем самые большие значения даже для минимума. 
1 регион не подойдет как минимум из-за риска большего чем 2.5, который нужно избежать. 3 тоже может подойти, но он не так прибылен